In [2]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone # only needed on colab as the computer is sometimes in a different timezone

In [7]:
def tomorrows_flight_arrivals(icao_list):

  #today = datetime.now() # all that's needed when working locally 
  today = datetime.now().astimezone(timezone('Europe/Berlin')).date() # only needed on colab as the computer is sometimes in a different timezone
  tomorrow = (today + timedelta(days=1))

  list_for_df = []

  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': "e723d7487fmsh7fce19b65c2f2eep119c33jsneeee8c0c0fc9"
          }
      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

In [8]:
icaos = ['EHAM']
tomorrows_flight_arrivals(icaos).head()

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EHAM,2022-11-03 23:25+01:00,None,Málaga,LEMG,2022-11-03 20:20+01:00,Transavia,HV 6118,2022-11-03
1,EHAM,2022-11-04 00:15+01:00,None,Marrakech,GMMX,2022-11-03 20:20+01:00,Transavia,HV 5752,2022-11-03
2,EHAM,2022-11-04 00:35+01:00,None,Amman,OJAI,None,Transavia,HV 6506,2022-11-03
3,EHAM,2022-11-04 00:40+01:00,None,Fuerteventura Island,GCFV,2022-11-03 19:05+00:00,Transavia,HV 6706,2022-11-03
4,EHAM,2022-11-04 05:30+01:00,None,Lagos,DNMM,None,Air France,AF 5479,2022-11-03
